**Pergunta :** O nivel de escolaridade afeta a chance de uma pessoa perder o emprego?

In [2]:
%matplotlib inline
import pandas as pd

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
desemprego_por_escolaridade = pd.read_csv("oecd.csv")
desemprego_por_escolaridade.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,1997,16.671282,NaN
1,DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,1998,16.517719,NaN
2,DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,1999,15.599174,NaN
3,DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,2000,13.665389,NaN
4,DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,2001,13.515862,NaN


In [4]:
desemprego= pd.read_csv("desemprego.csv")
desemprego.head(3)

,Country Name,Country Code,Series Name,Series Code,1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],...,2004 [YR2004],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,"Korea, Rep.",KOR,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,2.608,6.963,6.342,4.400,4.000,3.300,...,3.700,3.700,3.400,3.200,3.100,3.500,3.600,3.700,3.700,3.848
1,United States,USA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,5.000,4.510,4.219,3.992,4.731,5.783,...,5.529,9.633,8.949,8.069,7.375,6.168,5.280,4.869,4.355,3.896
2,Germany,DEU,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,9.863,9.788,8.855,7.917,7.773,8.482,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384


In [5]:
#Cruza as duas tabelas via país
escolaridade_index =desemprego_por_escolaridade.set_index('LOCATION')
desempregoT_index = desemprego.set_index('Country Code')

df = escolaridade_index.join(desempregoT_index, how='inner', lsuffix='_escolaridade', rsuffix='_desempregoT')
df.head()


,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes,Country Name,Series Name,Series Code,...,2004 [YR2004],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,1997,16.671282,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384
DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,1998,16.517719,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384
DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,1999,15.599174,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384
DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,2000,13.665389,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384
DEU,UNEMPEDU,BUPPSRY,PC_25_64,A,2001,13.515862,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384


In [9]:
df.rename(columns={'SUBJECT':'Escolaridade'},index={'DEU':'Alem','KOR':'Coreia','USA':'EUA'}, inplace=True)
df["Escolaridade"]= df["Escolaridade"].str.replace("BUPPSRY", "Prim_sem_Secun", case = False)
df["Escolaridade"]=df["Escolaridade"].str.replace("UPPSRY_NTRY", "Com_Secun_sem_superior", case = False) 
df["Escolaridade"]=df["Escolaridade"].str.replace("TRY", "Com_ens_superior", case = False) 
df.head()

,INDICATOR,Escolaridade,MEASURE,FREQUENCY,TIME,Value,Flag Codes,Country Name,Series Name,Series Code,...,2004 [YR2004],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
Alem,UNEMPEDU,Prim_sem_Secun,PC_25_64,A,1997,16.671282,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384
Alem,UNEMPEDU,Prim_sem_Secun,PC_25_64,A,1998,16.517719,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384
Alem,UNEMPEDU,Prim_sem_Secun,PC_25_64,A,1999,15.599174,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384
Alem,UNEMPEDU,Prim_sem_Secun,PC_25_64,A,2000,13.665389,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384
Alem,UNEMPEDU,Prim_sem_Secun,PC_25_64,A,2001,13.515862,NaN,Germany,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,...,10.727,6.966,5.824,5.379,5.231,4.981,4.624,4.122,3.746,3.384
